In [1]:
import easytrader
import pandas as pd
import datetime
from WindPy import w
import requests
import datetime
from Portfolio import portfolio
import time
import matplotlib.pyplot as plt

In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [4]:
position = {
    "160325.SZ":{
        "price":w.wsq('160325.SZ', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":0.857534
    }, 
    "161040.SZ":{
        "price":w.wsq('161040.SZ', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":0.90137
    }, 
    "501082.SH":{
        "price":w.wsq('501082.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":0.947945
    }, 
    "506002.SH":{
        "price":w.wsq('506002.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":0.986301
    },
    "160926.SZ":{
        "price":w.wsq('160926.SZ', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":0.945205
    }
}

fee = 0.2/100

naive = portfolio(cash = 1000000, position = position, cost = fee)

In [9]:
def naive1(interval, times, portfolio):
    
    cur_times = 0
    
    log_df = pd.DataFrame()
    
    while cur_times != times:
        
        now = datetime.datetime.now()
        print(now)
        
        naive.get_balance()
        
        # 1. create snapshot to document price and NAV
        snapshot = {now : {}}
        for ticker, info in portfolio.position.items():
            
            code = ticker.split('.')[0]
            headers = {
                'content-type': 'application/json',
                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:22.0) Gecko/20100101 Firefox/22.0'
            }
            response = requests.get(f'https://fundgz.1234567.com.cn/js/{code}.js', headers=headers)
            NAV = float(eval(response.text.replace('jsonpgz(', '').replace(');', ''))['gsz'])
            
            Price = w.wsq(ticker, 'rt_last').Data[0][0]
            info['price'] = Price
            
            AnnualizedRateOfReturn = ((NAV/Price)**(1/info['expiration'])-1)*100
            
            #NAV, Price, AnnualizedRateOfReturn
            snapshot[now][ticker + ' NAV'] = NAV
            snapshot[now][ticker + ' Price'] = Price
            snapshot[now][ticker + ' AnnualizedRateOfReturn'] = AnnualizedRateOfReturn
            
        tickerWithMaxAnnualizedRateOfReturn = max([(v, k) for k, v in snapshot[now].items() if 'AnnualizedRateOfReturn' in k])[1].split(' ')[0]
        print(tickerWithMaxAnnualizedRateOfReturn)
        
        # 2. Trade
        trade = 0
        for k, v in snapshot[now].items():
            print(k, v)
        
        min_benchmark = 100
        zero_holding_ticker = None
        
        for ticker, info in portfolio.position.items():
            
            if snapshot[now][ticker + ' AnnualizedRateOfReturn'] < min_benchmark and info['shares'] != 0:
                
                min_ticker = ticker
                min_benchmark = snapshot[now][ticker + ' AnnualizedRateOfReturn']
            
            if info['shares'] == 0:
                
                zero_holding_ticker = ticker
                
        if snapshot[now][zero_holding_ticker + ' AnnualizedRateOfReturn'] - snapshot[now][min_ticker + ' AnnualizedRateOfReturn'] >= 1:
            
            trade = 1
            sell_cost = naive.position[min_ticker]['shares'] * naive.position[min_ticker]['price'] * fee
            naive.sell(min_ticker, naive.position[min_ticker]['shares'], sell_cost)
            
        snapshot[now]['Capital'] = naive.get_capital()
        naive.get_balance()
        buy_cost = naive.cash * fee
        buy_shares = round((naive.cash - buy_cost)/snapshot[now][zero_holding_ticker + ' price'], -3)
        naive.buy(zero_holding_ticker, buy_shares, buy_cost)
          
            
        snapshot[now]['Capital'] = naive.get_capital()
        snapshot[now]['Trade'] = trade
        log_df = log_df.append(pd.DataFrame(snapshot).T)
            
        naive.get_balance()
        
        cur_times += 1
        time.sleep(interval)
        print('--'*50)
    
    return log_df

In [17]:
log_df = naive1(60, 10, naive)

2021-08-06 09:30:47.806203
Cash: 1000000
160325.SZ: 
      Price:   0.0
      Shares     0
161040.SZ: 
      Price:   0.0
      Shares     0
501082.SH: 
      Price:   0.0
      Shares     0
506002.SH: 
      Price:   0.0
      Shares     0
160926.SZ: 
      Price:   0.0
      Shares     0
Capital: 1000000.0
160325.SZ
160325.SZ NAV 1.3398
160325.SZ Price 1.175
160325.SZ AnnualizedRateOfReturn 16.539224702059663
161040.SZ NAV 1.5169
161040.SZ Price 1.375
161040.SZ AnnualizedRateOfReturn 11.511993731499448
501082.SH NAV 2.4392
501082.SH Price 2.191
501082.SH AnnualizedRateOfReturn 11.986141292171148
506002.SH NAV 1.3399
506002.SH Price 1.17
506002.SH AnnualizedRateOfReturn 14.73724460640251
160926.SZ NAV 1.3901
160926.SZ Price 1.216
160926.SZ AnnualizedRateOfReturn 15.207655394994802


UnboundLocalError: local variable 'min_ticker' referenced before assignment

In [12]:
from multiprocessing import Process

In [15]:
import multiprocessing
multiprocessing.cpu_count()

4